In [7]:
# ------------------------------------------------
# fetch by actress list by onejav
# ------------------------------------------------
# fetch_onejav_by_actress_list()

# ------------------------------------------------
# fetch by actress list by javland
# ------------------------------------------------
fetch_javland_by_actress_list()

# ------------------------------------------------
# generate full text search metadata for each movie
# ------------------------------------------------
# generate_fulltext()

[2024-08-09 13:38:13,060 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress AIKA
[2024-08-09 13:38:13,436 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress 篠田あゆみ
[2024-08-09 13:38:13,801 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress 枢木あおい
[2024-08-09 13:38:14,172 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress 大沢佑香
[2024-08-09 13:38:14,546 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress 佐々木あき
[2024-08-09 13:38:14,915 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress 上原亜衣
[2024-08-09 13:38:15,335 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress 吉沢明歩
[2024-08-09 13:38:15,695 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress 新村あかり
[2024-08-09 13:38:16,071 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress あべみかこ
[2024-08-09 13:38:16,444 - INFO - /tmp/ipykernel_2358480/381813712.py:65 - jav] - save actress 

KeyboardInterrupt: 

In [2]:
import asyncio
import json
import logging
import re
import sys
import time
import traceback
from collections import namedtuple
from concurrent.futures import ALL_COMPLETED, ThreadPoolExecutor, wait
from datetime import datetime
from hashlib import sha256
from io import BytesIO
from queue import Queue
from textwrap import dedent
from typing import Generator, List, NamedTuple, Optional, Union

import bson.json_util
import pymongo
import requests
from bs4 import BeautifulSoup
from bson import ObjectId
from kipp.decorator import timer
from kipp.utils import setup_logger
from minio import Minio
from pymongo import MongoClient
from tweepy import API, OAuthHandler

logger = setup_logger("jav")
logger.setLevel(logging.INFO)
executor = ThreadPoolExecutor(max_workers=100)


sys.path.append(r"/opt/configs/ramjet")
import prd

# mongodb
mongo = MongoClient(
    f"mongodb://{prd.MONGO_ADMIN_USER}:{prd.MONGO_ADMIN_PASSWD}@{prd.MONGO_HOST}:{prd.MONGO_PORT}",
)
col_actress = mongo["jav"]["actress"]
col_movies = mongo["jav"]["movies"]
col_fulltext = mongo["jav"]["fulltext"]


# add index to db
col_movies.create_index([("name", pymongo.ASCENDING)])
col_fulltext.create_index([("word", pymongo.ASCENDING)], unique=True)

# minio
s3cli: Minio = Minio(
    endpoint=prd.S3_MINIO_ADDR,
    access_key=prd.S3_KEY,
    secret_key=prd.S3_SECRET,
    secure=True,
)


class Actress(NamedTuple):
    id: ObjectId
    name: str
    other_names: List[str]
    url: str


class Movie(NamedTuple):
    actresses: List[ObjectId]
    name: str
    description: Optional[str]
    img_urls: List[str]
    tags: List[str]

In [3]:
# =========================================
# https://onejav.com/
# =========================================

actress_index_url = "https://onejav.com/actress/"

REG_ONEJAV_ACTRESS_NAME = re.compile(r"([\w ]+)(?:<[^>]+>(\w+).*)?")


@timer
def gen_onejav_actress(q: Queue[Optional[Actress]]):
    n_page = 1
    n_retry = 0
    while True:
        url = f"https://onejav.com/actress/?page={n_page}"
        resp = requests.get(url)
        if resp.status_code == 500:
            if n_retry < 3:
                time.sleep(3)
                logger.warn(f"gen_onejav_actress retry {url=}, {n_retry=}")
                n_retry += 1
                continue
            else:
                n_retry = 0
                n_page += 1
                continue
        elif resp.status_code == 404:
            logger.info(f"gen_onejav_actress exit since for {url=}")
            return
        elif resp.status_code != 200:
            logger.info(
                f"gen_onejav_actress exit for {url=} [{resp.status_code}]{resp.text}"
            )
            return

        soup = BeautifulSoup(resp.text, "html.parser")
        for card in soup.select(".container .card"):
            name, url = "", ""

            ele = card.select_one(".card-header .card-header-title")
            assert ele, f"cannot find actress name for {url=}"
            name = ele.decode_contents().strip()

            ele = card.select_one("a")
            assert ele, f"cannot find actress url for {url=}"
            url = f"https://onejav.com{ele.attrs['href']}"

            # parse actress name
            # name = 'Jun Suehiro <small class="text-muted ml-1">末広純</small>'
            other_names = []
            [(alt_name, name)] = REG_ONEJAV_ACTRESS_NAME.findall(name)
            name = name.strip()
            alt_name = alt_name.strip()
            name = name or alt_name

            logger.debug(f"yield actress {name}")
            actress = save_actress(name, url, list(set([name, alt_name])))
            q.put(actress)

        n_page += 1


def save_actress(name: str, url: str, other_names: List[str]) -> Actress:
    """save actress to db, return actress_id"""
    logger.info(f"save actress {name}")

    col_actress.update_one(
        {"name": name},
        {
            "$set": {
                "name": name,
                "url": url,
                "other_names": other_names,
            }
        },
        upsert=True,
    )

    docu = col_actress.find_one({"name": name})
    assert docu, f"can not find actress {name}"
    return Actress(
        name=name,
        url=url,
        id=docu["_id"],
        other_names=docu["other_names"],
    )


def replace_image_url(img_url) -> str:
    """download image and upload to s3, return new url"""
    resp = requests.get(img_url)
    assert resp.status_code == 200

    img_content = resp.content
    digest = sha256(img_content).hexdigest()
    objkey = f"jav/{digest[:2]}/{digest[2:4]}/{digest}.png"

    # check whether image exists
    new_img_url = f"{prd.S3_SERVER}/public/{objkey}"
    if requests.head(new_img_url).status_code == 200:
        return new_img_url

    logger.info(f"upload image to s3: {objkey}")
    s3cli.put_object(
        bucket_name="public",
        object_name=objkey,
        data=BytesIO(img_content),
        length=len(img_content),
        content_type="image/png",
    )

    return new_img_url


@timer
def gen_onejav_movies_by_actress(upstream_q: Queue[Optional[Actress]]):
    n_page = 1
    n_retry = 0
    while True:
        actress = upstream_q.get()
        if actress is None:
            upstream_q.put(None)
            logger.info(f"gen_onejav_movies_by_actress exit")
            return

        url = f"{actress.url}?page={n_page}"
        resp = requests.get(url)
        if resp.status_code == 500:
            if n_retry < 3:
                time.sleep(3)
                logger.warn(f"gen_movies retry {url=}, {n_retry=}")
                n_retry += 1
                continue
            else:
                n_retry = 0
                n_page += 1
                continue
        elif resp.status_code == 404:
            logger.info(f"gen_movies exit since for {url=}")
            return
        elif resp.status_code != 200:
            logger.info(
                f"gen_movies exit since for {url=} [{resp.status_code}]{resp.text}"
            )
            return

        soup = BeautifulSoup(resp.text, "html.parser")
        for card in soup.select(".container .card"):
            try:
                ele = card.select_one(".image")
                assert ele, f"can not find image {url=}"
                img_url = ele.attrs["src"]

                ele = card.select_one(".title a")
                assert ele, f"can not find title {url=}"
                name = ele.decode_contents().strip()

                description = None
                if card.select_one(".has-text-grey-dark"):
                    ele = card.select_one(".has-text-grey-dark")
                    assert ele, f"can not find description {url=}"
                    description = ele.decode_contents().strip()
                tags = [
                    ele.decode_contents().strip() for ele in card.select(".tags .tag")
                ]

                img_url = replace_image_url(img_url)
                movie = Movie(
                    actresses=[actress.id],
                    name=name,
                    description=description,
                    img_urls=[img_url],
                    tags=tags,
                )
                save_movie(movie)
            except Exception:
                logger.exception(f"parse movie error {url=}")
                continue

        n_page += 1


REGEX_MOVIE_NAME = re.compile(r"^([A-Z]+)(\d+)$")


def save_movie(movie: Movie):
    movie_name = movie.name.upper().strip()
    matched = REGEX_MOVIE_NAME.findall(movie_name)
    if matched:
        prefix, num = matched[0]
        if prefix and num:
            movie_name = f"{prefix}-{num}"

    img_urls = [v.strip() for v in movie.img_urls if v.strip()]
    tags = [v.strip() for v in movie.tags if v.strip()]
    actresses = [v for v in movie.actresses if v]

    logger.info(f"save movie {movie_name}")
    col_movies.update_one(
        {"name": movie_name},
        {
            "$set": {
                "name": movie_name,
                "description": movie.description,
            },
            "$addToSet": {
                "actresses": {"$each": actresses},
                "tags": {"$each": tags},
                "img_urls": {"$each": img_urls},
            },
        },
        upsert=True,
    )


@timer
def fetch_onejav_by_actress_list():
    """fetch actresses and movies by actress list and save to db"""
    actress_queue = Queue(maxsize=10)

    fs_load_actress = []
    fs_load_movies = []

    fs_load_actress.append(executor.submit(gen_onejav_actress, actress_queue))
    for _ in range(20):
        fs_load_movies.append(
            executor.submit(gen_onejav_movies_by_actress, actress_queue)
        )

    wait(fs_load_actress, return_when=ALL_COMPLETED)
    actress_queue.put(None)
    wait(fs_load_movies, return_when=ALL_COMPLETED)

In [4]:
# =========================================
# https://jav.land/
# =========================================

actress_index_url = "https://jav.land/ja/star_list.php?prefix={alphabet}&page={page}"

REGEX_JAVLAND_ACTRESS_NAME = re.compile(r"(\w+)(?:（(\w+)(?:、(\w+){0,}）))?")


class JavlandMovieInfo(NamedTuple):
    actress: Actress
    url: str
    description: Optional[str]


@timer
def gen_javland_actress(q: Queue[Optional[Actress]]):
    n_page = 1
    alphabet = "A"
    n_retry = 0
    while alphabet <= "Z":
        while True:
            url = actress_index_url.format(alphabet=alphabet, page=n_page)
            try:
                resp = requests.get(url)
                if resp.status_code %100 == 5:
                    if n_retry < 3:
                        time.sleep(3)
                        logger.warn(f"gen_javland_actress retry {url=}, {n_retry=}")
                        n_retry += 1
                        continue
                    else:
                        n_retry = 0
                        n_page += 1
                        continue
                elif resp.status_code == 404:
                    logger.info(f"gen_javland_actress exit since for {url=}")
                    break
                elif resp.status_code != 200:
                    logger.info(
                        f"gen_javland_actress exit for {url=} [{resp.status_code}]{resp.text}"
                    )
                    break

                soup = BeautifulSoup(resp.text, "html.parser")
                cards = soup.select(".container-fluid .genre_list a.text-center")
                if not cards:
                    break

                for card in cards:
                    raw_name = card.decode_contents().strip()

                    # parse actress name
                    # name = "馬場ふみか" / ayami（赤西涼、まひる）
                    matched = REGEX_JAVLAND_ACTRESS_NAME.findall(raw_name)
                    assert matched, f"can not parse actress name {raw_name=}"
                    assert matched[0], f"can not parse actress name {raw_name=}"
                    other_names = list(
                        set([v.strip() for v in matched[0] if v.strip()])
                    )
                    name = other_names[0]
                    assert name, f"can not parse actress name {raw_name=}"

                    actress_url = f"https://jav.land{card.attrs['href']}"

                    logger.debug(f"yield actress {name}")
                    actress = save_actress(name, actress_url, other_names)
                    q.put(actress)
            except Exception:
                logger.exception(f"gen_javland_actress error {url=}")
                continue

            n_page += 1

        n_page = 1
        alphabet = chr(ord(alphabet) + 1)


@timer
def gen_javland_movies_by_actress(
    upstream_q: Queue[Optional[Actress]],
    downstream_q: Queue[Optional[JavlandMovieInfo]],
):
    while True:
        actress = upstream_q.get()
        if actress is None:
            upstream_q.put(None)
            logger.info(f"gen_javland_movies_by_actress exit")
            return

        n_page = 1
        n_retry = 0
        while True:
            url = f"{actress.url}?page={n_page}"
            try:
                resp = requests.get(url)
                if resp.status_code %100 == 5:
                    if n_retry < 3:
                        time.sleep(3)
                        logger.warn(f"gen_movies retry {url=}, {n_retry=}")
                        n_retry += 1
                        continue
                    else:
                        n_retry = 0
                        n_page += 1
                        continue
                elif resp.status_code == 404:
                    logger.info(f"gen_movies exit since for {url=}")
                    break
                elif resp.status_code != 200:
                    logger.warn(
                        f"gen_movies skip since for {url=} [{resp.status_code}]{resp.text}"
                    )
                    n_page += 1
                    continue

                soup = BeautifulSoup(resp.text, "html.parser")
                cards = soup.select(".container-fluid .videothumblist .video.panel")
                if not cards:
                    logger.info(f"can not find cards for {url=}")
                    break

                for card in cards:
                    ele = card.select_one(".panel-footer a")
                    assert ele, f"can not find url {url=}"
                    url = f"https://jav.land{ele.attrs['href']}"
                    description = ele.decode_contents().strip()

                    downstream_q.put(
                        JavlandMovieInfo(actress=actress, url=url, description=description)
                    )

                n_page += 1
            except:
                logger.exception(f"gen_javland_movies_by_actress got error for {url=}")


@timer
def fetch_javland_movie(q: Queue[Optional[JavlandMovieInfo]]):
    while True:
        movie_info = q.get()
        if movie_info is None:
            q.put(None)
            logger.info(f"fetch_javland_movie exit")
            return

        try:
            resp = requests.get(movie_info.url, headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            })
            if resp.status_code != 200:
                logger.error(
                    f"fetch_javland_movie error, {movie_info.url=} [{resp.status_code}]{resp.text}"
                )
                continue

            soup = BeautifulSoup(resp.text, "html.parser")

            ele = soup.select_one(".k-right img.img-responsive")
            assert ele, f"can not find image {movie_info.url=}"
            img_url = ele.attrs["src"]
            img_url = replace_image_url(img_url)

            name = ""
            tags = []
            rows = soup.select(".k-right .videotextlist tr")
            for row in rows:
                ele = row.select_one("td:nth-child(1)")
                if ele:
                    key = ele.decode_contents().strip()
                else:
                    continue

                ele = row.select_one("td:nth-child(2)")
                if ele:
                    value = ele.decode_contents().strip()
                else:
                    continue

                if "DVD ID" in key:
                    name = value
                elif "ジャンル" in key:
                    vals = row.select("td:nth-child(2) .genre a")
                    tags = list(set([val.decode_contents().strip() for val in vals]))

            movie = Movie(
                actresses=[movie_info.actress.id],
                name=name,
                description=movie_info.description,
                img_urls=[img_url],
                tags=tags,
            )
            save_movie(movie)
        except Exception:
            logger.exception(f"fetch_javland_movie error {movie_info.url=}")
            continue


@timer
def fetch_javland_by_actress_list():
    """fetch actresses and movies by actress list and save to db"""
    actress_queue = Queue(maxsize=10)
    movie_queue = Queue(maxsize=10)

    fs_load_actress = []
    fs_load_movies = []
    fs_save_movies = []

    fs_load_actress.append(executor.submit(gen_javland_actress, actress_queue))
    for _ in range(20):
        fs_load_movies.append(
            executor.submit(gen_javland_movies_by_actress, actress_queue, movie_queue)
        )
        fs_save_movies.append(executor.submit(fetch_javland_movie, movie_queue))

    wait(fs_load_actress, return_when=ALL_COMPLETED)
    actress_queue.put(None)
    wait(fs_load_movies, return_when=ALL_COMPLETED)
    movie_queue.put(None)
    wait(fs_save_movies, return_when=ALL_COMPLETED)

In [5]:
def llm_oneshot(model, system_prompt, user_prompt) -> str:
    """get response from LLM"""
    url = f"{prd.OPENAI_API}/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {prd.OPENAI_TOKEN}",
    }
    body = {
        "model": model,
        "max_tokens": 3000,
        "stream": False,
        "messages": [
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
    }
    resp = requests.post(url, headers=headers, json=body)
    assert resp.status_code == 200, f"[{resp.status_code}]{resp.text}"

    return resp.json()["choices"][0]["message"]["content"]


REGEX_SEGMENT_RESULT = re.compile(r"(?:```json)?(.+\\])(?:.*)")


def segment_word(sentence: str) -> List[str]:
    """segment sentence to words"""
    system_prompt = dedent(
        """
        You play the role of a tokenization engine. I will provide you with a passage,
        and after reading it carefully, you will return the most important nouns in the form of a JSON array,
        formatted as ["word", "word"]. your response should be a valid JSON string,
        do not contains any other characters.
        """
    )
    user_prompt = sentence

    resp = llm_oneshot("deepseek-coder", system_prompt, user_prompt)
    matched = REGEX_SEGMENT_RESULT.findall(resp)
    assert len(matched) > 1, f"can not find segment result {resp}"

    words = json.loads(matched[0][0])
    assert isinstance(words, list), f"invalid response {words}"

    words = [word.strip() for word in words if isinstance(word, str) and word.strip()]
    words = list(set(words))

    return words


@timer
def _list_movies_in_db(q: Queue[Movie]):
    for docu in col_movies.find():
        movie = Movie(
            actresses=docu["actresses"],
            name=docu["name"],
            description=docu["description"],
            img_urls=docu["img_urls"],
            tags=docu["tags"],
        )
        q.put(movie)


class FulltextItem(NamedTuple):
    movie_id: ObjectId
    word: str


@timer
def _generate_fulltext_for_movie(
    upstream_q: Queue[Optional[Movie]], downstream_q: Queue[FulltextItem]
):
    """generate fulltext for each movie, each word points to a movie"""
    while True:
        try:
            movie = upstream_q.get()
            if movie is None:
                upstream_q.put(None)
                logger.info(f"_generate_fulltext_for_movie exit")
                return

            # load movie
            logger.info(f"_generate_fulltext_for_movie for {movie.name=}")
            movie_docu = col_movies.find_one({"name": movie.name})
            assert movie_docu, f"can not find movie {movie.name}"

            # load actress
            actress_summary = ""
            for docu in col_actress.find({"_id": {"$in": movie_docu["actresses"]}}):
                actress_summary += f'{docu.get("name", "")}, {docu.get("other_names", []).join(", ")}, '

            movie_summary = f'{actress_summary}, {movie.name}, {movie.description}'
            words = segment_word(movie_summary)
            for word in words:
                downstream_q.put(FulltextItem(movie_id=movie_docu["_id"], word=word))
        except Exception:
            logger.exception(f"error in _generate_fulltext_for_movie")


@timer
def _save_fulltext_item(q: Queue[Optional[FulltextItem]]):
    """save fulltext item to db, each word may points to multiple movies"""
    while True:
        try:
            item = q.get()
            if item is None:
                q.put(None)
                logger.info(f"_save_fulltext_item exit")
                return

            # each word may points to multiple movies,
            # these movies are stored in a set
            col_fulltext.update_one(
                {"word": item.word},
                {"$addToSet": {"movies": item.movie_id}},
                upsert=True,
            )
        except Exception:
            logger.exception(f"error in _save_fulltext_item")


@timer
def generate_fulltext():
    movie_queue = Queue(maxsize=10)
    fulltext_queue = Queue(maxsize=10)

    fs_list_movies = []
    fs_generate_fulltext = []
    fs_save_fulltext = []

    fs_list_movies.append(executor.submit(_list_movies_in_db, movie_queue))
    for i in range(20):
        fs_generate_fulltext.append(
            executor.submit(_generate_fulltext_for_movie, movie_queue, fulltext_queue)
        )
        fs_save_fulltext.append(executor.submit(_save_fulltext_item, fulltext_queue))

    wait(fs_list_movies, return_when=ALL_COMPLETED)
    movie_queue.put(None)
    wait(fs_generate_fulltext, return_when=ALL_COMPLETED)
    fulltext_queue.put(None)
    wait(fs_save_fulltext, return_when=ALL_COMPLETED)

In [6]:
REGEX_MOVIE_NAME = re.compile(r"^([A-Z]+)(\d+)$")


def convert_movies():
    # remove empty images
    for docu in col_movies.find({"img_urls": ""}):
        imgs = [v for v in docu["img_urls"] if v.strip()]
        col_movies.update_one(
            {"_id": docu["_id"]},
            {"$set": {"img_urls": imgs}},
        )


# convert_movies()